<a href="https://colab.research.google.com/github/TNN-A/us-ie-big-data-technologies/blob/master/Copy_of_Apache_Beam_Interactive_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive Beam

In this notebook, we set up your development environment and work through a simple example using the [DirectRunner](https://beam.apache.org/documentation/runners/direct/). You can explore other runners with the [Beam Capatibility Matrix](https://beam.apache.org/documentation/runners/capability-matrix/).

The expectation is that this notebook will help you explore the tutorial in a more interactive way.

To learn more about Colab, see [Welcome to Colaboratory!](https://colab.sandbox.google.com/notebooks/welcome.ipynb).

# Setup

First, you need to set up your environment, which includes installing `apache-beam` and downloading a text file from Cloud Storage to your local file system. We are using this file to test your pipeline.

In [ ]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

run('pip install --upgrade pip')

# Install apache-beam.
run('pip install --quiet apache-beam')

# Copy the input file into the local file system.
run('mkdir -p data')
run('gsutil cp gs://dataflow-samples/shakespeare/kinglear.txt data/')

In [ ]:
! wc -l data/kinglear.txt


In [ ]:

! head -3 data/kinglear.txt

In [ ]:
import apache_beam as beam
import re

inputs_pattern = 'data/*'
outputs_prefix = 'outputs/part'

## How to interactively work with Beam

Here is an example of how to work iteratively with beam in order to understand what is happening in your pipeline.

Firstly, reduce the size of the King Lear file to be manageable

In [ ]:

! head -10 data/kinglear.txt > data/small.txt
! wc -l data/small.txt

Create a custom print function (the python3 function `print` is supposed to work but we define our own here). Then it is possible to see what you are doing to the file.

But something is wrong... why is it printing twice, see [SO](https://stackoverflow.com/a/52282001/1185293)

In [ ]:
def myprint(x):
  print('{}'.format(x))

with beam.Pipeline() as pipeline:
  (pipeline
      | 'Read lines' >> beam.io.ReadFromText('data/small.txt')
      | "print" >> beam.Map(myprint)
  )

result = pipeline.run()
result.wait_until_finish()

Now, let's break split each line on spaces and get the words out.

In [ ]:

with beam.Pipeline() as pipeline:
  (pipeline
      | 'Read lines' >> beam.io.ReadFromText('data/small.txt')
      | 'get words' >> beam.FlatMap(lambda line: re.findall(r"[a-zA-Z']+", line))
      | "print" >> beam.Map(myprint)
  )

Recall that `flatMap`s typically act on something (a function, iterable or variable) and apply a function to that something to produce a list of elements. See [this](https://beam.apache.org/documentation/transforms/python/elementwise/flatmap/) great example of how FlatMap works in Beam, and this answer on [SO](https://stackoverflow.com/a/45682977/1185293) for a simple explanation.

In the case above, we applied an anonymous function (lambda function) to a line. We can define it explicitly if you prefer a more conventional syntax

In [ ]:
def my_line_split_func(line):
  return re.findall(r"[a-zA-Z']+", line)

with beam.Pipeline() as pipeline:
  (pipeline
      | 'Read lines' >> beam.io.ReadFromText('data/small.txt')
      | 'get words' >> beam.FlatMap(my_line_split_func)
      | "print" >> beam.Map(myprint)
  )


### Tutorial



In [ ]:
! echo -e 'r1c1,r1c2,2020/03/05\nr2c1,r2c2,2020/03/23' > data/play.csv


In [ ]:

class Transform(beam.DoFn):

  # Use classes to perform transformations on your PCollections
  # Yield or return the element(s) needed as input for the next transform
  def process(self, element):
    yield element


with beam.Pipeline() as pipeline:
  (pipeline
      | 'Read lines' >> beam.io.ReadFromText('data/play.csv')
      | 'format line' >> beam.ParDo(Transform())
      | "print" >> beam.Map(myprint)
  )


result.wait_until_finish()

In [ ]:
! cat users_v.csv | head

! cat orders_v_2022.csv | head

user_id,name,gender,age,address,date_joined
1,Anthony Wolf,male,73,New Rachelburgh-VA-49583,2019/03/13
2,James Armstrong,male,56,North Jillianfort-UT-86454,2020/11/06
3,Cody Shaw,male,75,North Anne-SC-53799,2004/05/29
4,Sierra Hamilton,female,76,New Angelafurt-ME-46190,2005/08/26
5,Chase Davis,male,31,South Bethmouth-WI-18562,2018/04/30
6,Sierra Andrews,female,21,Ryanville-MI-69690,2007/05/25
7,Ann Stone,female,41,Smithmouth-SD-17340,2005/01/05
8,Karen Santos,female,34,Mariaville-AK-29888,2003/12/12
9,Ronald Meyer,male,41,North Cherylhaven-NJ-04197,2015/11/14
order_no;user_id;product_list;date_purchased
1000;1887;Cassava;2000-01-01
1001;838;Calabash, Water Spinach;2000-01-01
1002;2032;Onion, Rapini;2000-01-01
1003;1482;Swiss Chard, Artichoke;2000-01-01
1004;475;Turnip Greens, Plantain;2000-01-01
1005;1627;English Cucumber, Parsley Root, Cauliflower;2000-01-01
1006;2000;Bell Pepper, English Cucumber;2000-01-01
1007;2099;Arugula;2000-01-01
1008;2337;Shallots, Jerusalem Artichoke;2000-01-

In [ ]:
#Question 4.1

with beam.Pipeline() as pipeline:
  users_pairs = pipeline | 'Create users' >> beam.Create(['users_v.csv'])



orders_pairs = pipeline | 'Create orders' >> beam.Create(['orders_v_2022.csv'])

files = (({'users': users_pairs, 'orders': orders_pairs}) | 'Merge' >> beam.CoGroupByKey() | beam.Map(print)
)





In [ ]:
#Question 4.2




with beam.Pipeline() as pipeline:
       (pipeline
           | 'Read lines' >> beam.io.ReadFromText('users_v.csv')
           | 'format line' >> beam.ParDo(Transform())
           | 'add_key' >> beam.Map(lambda elem: (elem[1], 1))  # emit (M/F, 1) pairs
           | "print" >> beam.Map(print)
)

('s', 1)
(',', 1)
(',', 1)
(',', 1)
(',', 1)
(',', 1)
(',', 1)
(',', 1)
(',', 1)
(',', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('0', 1)
('0', 1)
('0', 1)
('0', 1)
('0', 1)
('0', 1)
('0', 1)
('0', 1)
('0', 1)
('1', 1)
(

In [ ]:
#Question 4.3

import pandas as pd

df = pd.read_csv('users_v.csv', 'orders_v_2022.csv') df.gender.value_counts()

SyntaxError: invalid syntax (<ipython-input-101-c1f1146a541e>, line 5)

In [ ]:
#Question 4.4



('s', 1)
(',', 1)
(',', 1)
(',', 1)
(',', 1)
(',', 1)
(',', 1)
(',', 1)
(',', 1)
(',', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('1', 1)
('2', 1)
('3', 1)
('4', 1)
('5', 1)
('6', 1)
('7', 1)
('8', 1)
('9', 1)
('0', 1)
('0', 1)
('0', 1)
('0', 1)
('0', 1)
('0', 1)
('0', 1)
('0', 1)
('0', 1)
('0', 1)
('1', 1)
(

In [ ]:
#Question 4.5

In [ ]:
#Question 4.6

In [ ]:
#Question 4.7

